In [1]:
# Upload Database
from google.colab import files
uploaded = files.upload()

Saving winequality-red.csv to winequality-red.csv


In [2]:
import pandas as pd

df = pd.read_csv('winequality-red.csv')

In [3]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [16]:
# Buat daftar yang berisi kualitas wine
kualitas_wine = [3, 4, 5,6,7,8]

# Buat dictionary untuk menyimpan jumlah data pada setiap label
jumlah_data_per_label = {}

# Menggunakan perulangan untuk menghitung jumlah data pada setiap label
for label in kualitas_wine:
    jumlah_data = len(df[df['quality'] == label])
    jumlah_data_per_label[label] = jumlah_data

# Cetak jumlah data pada setiap label
for label, jumlah_data in jumlah_data_per_label.items():
    print(f"Jumlah data untuk kualitas {label}: {jumlah_data}")


Jumlah data untuk kualitas 3: 10
Jumlah data untuk kualitas 4: 53
Jumlah data untuk kualitas 5: 681
Jumlah data untuk kualitas 6: 638
Jumlah data untuk kualitas 7: 199
Jumlah data untuk kualitas 8: 18


In [20]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
# Pisahkan atribut (fitur) dan label (kualitas wine)
X = df.drop('quality', axis=1)  # Atribut
y = df['quality']  # Label
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [21]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

# Bagi dataset menjadi data pelatihan (80%) dan data pengujian (20%)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

In [24]:
from sklearn.model_selection import GridSearchCV

# Tentukan daftar nilai yang mungkin untuk setiap hyperparameter
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
}

# Inisialisasi model yang akan Anda optimalkan
model = RandomForestClassifier(random_state=42)

# Lakukan grid search
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Cetak parameter terbaik yang ditemukan
print("Parameter terbaik:", grid_search.best_params_)

Parameter terbaik: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}


In [38]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Inisialisasi model Random Forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=20, min_samples_split=2, min_samples_leaf=1, max_features='auto', random_state=42)


# Latih model pada data pelatihan
rf_model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


RandomForestClassifier(max_depth=20, max_features='auto', n_estimators=200,
                       random_state=42)

In [39]:
# Memprediksi data pengujian
y_pred = rf_model.predict(X_test)

# Mengukur akurasi
accuracy = accuracy_score(y_test, y_pred)
print(f'Akurasi: {accuracy}')

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))

# Menampilkan matriks kebingungan (confusion matrix)
confusion = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:')
print(confusion)

Akurasi: 0.8814180929095354
              precision    recall  f1-score   support

           3       0.96      1.00      0.98       136
           4       0.92      0.97      0.94       136
           5       0.80      0.77      0.78       136
           6       0.75      0.62      0.68       137
           7       0.87      0.93      0.90       137
           8       0.97      1.00      0.99       136

    accuracy                           0.88       818
   macro avg       0.88      0.88      0.88       818
weighted avg       0.88      0.88      0.88       818

Confusion Matrix:
[[136   0   0   0   0   0]
 [  0 132   3   1   0   0]
 [  3   6 105  21   1   0]
 [  1   6  24  85  18   3]
 [  2   0   0   7 127   1]
 [  0   0   0   0   0 136]]
